### Bibliotecas

In [1]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import re
from unidecode import unidecode

### Imóveis comum

In [2]:
# URL inicial
start_url = "https://www.zapimoveis.com.br/venda/apartamentos/pr+cascavel/?__ab=exp-aa-test:control,propposv3:control,rentsale:control,pos-zap:new,zapproppos:a&transacao=venda&onde=,Paran%C3%A1,Cascavel,,,,,city,BR%3EParana%3ENULL%3ECascavel,-24.954136,-53.480241,&tipos=apartamento_residencial,studio_residencial,kitnet_residencial,casa_residencial,sobrado_residencial,condominio_residencial,casa-vila_residencial,cobertura_residencial,flat_residencial,loft_residencial&pagina=1"

In [3]:
def accept_cookies(driver):
    """Aceita cookies caso o banner apareça."""
    try:
        cookies_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceitar todos') or contains(text(), 'Aceitar')]"))
        )
        cookies_button.click()
        print("Cookies aceitos.")
    except TimeoutException:
        print("Banner de cookies não encontrado ou já aceito.")

In [4]:
def scroll_a_little(driver):
    """Rola a página levemente para carregar novos elementos."""
    print("Rolando um pouco a página...")
    driver.execute_script("window.scrollBy(0, 280);")  # Rolagem menor
    time.sleep(2)  # Tempo menor de espera

In [5]:
def extract_id_from_url(url):
    """
    Extrai o ID do imóvel do URL.
    """
    # Remove a barra final, se existir, e pega o texto após o último hífen
    return url.rstrip('/').split('-')[-1]

In [20]:
def scrape_property_data(driver, property_id, ad_text):
    """Coleta os dados de um imóvel na aba atual."""
    data = {"ID": property_id}  # Adicionar ID como chave primária

    # Coletar características principais
    try:
        features = [feature.text for feature in driver.find_elements(By.CSS_SELECTOR, "span[data-cy='ldp-propertyFeatures-txt']")]
        data['Área'] = features[0] if len(features) > 0 else "Não encontrado"
        data['Quartos'] = features[1] if len(features) > 1 else "Não encontrado"
        data['Banheiros'] = features[2] if len(features) > 2 else "Não encontrado"
        data['Vagas de Carro'] = features[3] if len(features) > 3 else "Não encontrado"
    except Exception:
        data['Área'] = data['Quartos'] = data['Banheiros'] = data['Vagas de Carro'] = "Não encontrado"

    # Coletar preço
    try:
        meta_description = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "meta[name='description']"))
        )
        description_content = meta_description.get_attribute("content")
        price_match = re.search(r"R\$ [\d.,]+", description_content)
        data['Preço'] = price_match.group(0) if price_match else "Não encontrado"
    except TimeoutException:
        data['Preço'] = "Não encontrado"

    # Coletar endereço
    try:
        address_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.address-info-value"))
        )
        data['Endereço'] = address_element.text
    except TimeoutException:
        data['Endereço'] = "Não encontrado"

    # Coletar itens adicionais
    try:
        additional_items = [item.text for item in driver.find_elements(By.CSS_SELECTOR, "span.amenities-item-text[data-cy='ldp-propertyFeatures-txt']")]
        data['Itens Adicionais'] = ", ".join(additional_items) if additional_items else "Não encontrado"
    except Exception:
        data['Itens Adicionais'] = "Não encontrado"

    # Coletar códigos do anunciante e Zap
    try:
        codes_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p[data-cy='ldp-propertyCodes-txt']"))
        )
        data['Códigos'] = codes_element.text
    except TimeoutException:
        data['Códigos'] = "Não encontrado"

    # Usar o texto do anúncio coletado previamente
    data['Texto do Anúncio'] = ad_text

    return data

In [7]:
def click_next_page(driver):
    """Clica no botão de próxima página, se disponível."""
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='next-page']"))
        )
        next_button.click()
        time.sleep(2)
        print("Avançando para a próxima página...")
        return True
    except TimeoutException:
        print("Botão de próxima página não encontrado. Fim da navegação.")
        return False

In [8]:
def save_progress(processed_ids):
    """Salva os IDs processados para retomada em caso de falha."""
    with open("progress.txt", "w") as file:
        file.write("\n".join(processed_ids))

In [9]:
def load_progress():
    """Carrega os IDs processados de um arquivo, se existir."""
    try:
        with open("progress.txt", "r") as file:
            return set(file.read().splitlines())
    except FileNotFoundError:
        return set()

### OK

In [11]:
def scroll_to_end(driver):
    """
    Rola gradualmente até o final da página com base no carregamento dinâmico de anúncios.
    Usa o elemento <img> especificado para determinar o final da página.
    """
    while True:
        try:
            # Verifica se o elemento <img> está presente na página
            footer_image = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "img[src='https://cdn-zap-ssr-prod.zapimoveis.com.br/_next/public/assets/images/reclameaqui-stamp.svg']"))
            )
            if footer_image.is_displayed():
                print("Elemento do rodapé encontrado. Final da página alcançado.")
                break
        except Exception:
            print("Elemento do rodapé não encontrado ainda. Continuando a rolagem...")

        # Rola um pouco para baixo
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.7)  # Aguarda carregamento de novos elementos

    # Volta ao topo da página
    print("Retornando ao topo da página...")
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)

In [11]:
def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    columns = ["ID", "Códigos", "Área", "Quartos", "Banheiros", "Vagas de Carro", "Preço", "Endereço", "Itens Adicionais", "Texto do Anúncio"]
    df = pd.DataFrame(columns=columns)
    processed_ids = load_progress()  # Carrega IDs processados de arquivo
  
    try:
        driver.get(start_url)
        accept_cookies(driver)

        # Rola até o final da página para carregar todos os imóveis
        print("Rolando gradualmente para carregar todos os anúncios...")
        scroll_to_end(driver)

        # Inicia a coleta dos imóveis
        listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
        while listings: 
            for i in range(len(listings)):
                try:
                    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
                    link = listings[i].find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    property_id = extract_id_from_url(link)

                    if property_id in processed_ids:
                        print(f"Imóvel já processado: ID {property_id}")
                        continue

                    # Coletar texto do anúncio dentro do contêiner
                    try:
                        container = listings[i].find_element(By.XPATH, "./ancestor::div[contains(@class, 'BaseCard_card__content__pL2Vc')]")
                        description_element = container.find_element(By.CSS_SELECTOR, "p.ListingCard_card__description__slBTG[data-cy='rp-cardProperty-description-txt']")
                        raw_text = description_element.text if description_element else "Não encontrado"
                        ad_text = unidecode(raw_text)  # Remove inconsistências de codificação
                    except Exception as e:
                        print(f"Erro ao coletar texto do anúncio no imóvel {i + 1}: {e}")
                        ad_text = "Não encontrado"

                    print(f"\nProcessando imóvel {i + 1} - ID: {property_id}")
                    print(f"Texto do Anúncio: {ad_text[:100]}...")  # Mostra uma parte do texto para depuração

                    processed_ids.add(property_id)  # Adiciona ID ao conjunto de processados
                    save_progress(processed_ids)  # Salva progresso continuamente

                    start_time = time.time()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listings[i])).click()
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])
                    accept_cookies(driver)

                    data = scrape_property_data(driver, property_id, ad_text)
                    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    elapsed_time = time.time() - start_time
                    print(f"Tempo entre clique e retorno: {elapsed_time:.2f} segundos")

                    scroll_a_little(driver)
                except Exception as e:
                    print(f"Erro ao processar imóvel: {e}")

            # Tenta avançar para a próxima página
            if not click_next_page(driver):
                break  # Sai do loop caso não haja mais páginas

            # Atualiza a lista de imóveis para a nova página
            listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")

    finally:
        driver.quit()
        df.to_csv("imoveis_zapimoveis.csv", index=False, encoding="utf-8")
        print("Dados salvos no arquivo 'imoveis_zapimoveis.csv'.")

if __name__ == '__main__':
    scrape_zapimoveis()

Cookies aceitos.
Imóvel já processado: ID 2717453185

Processando imóvel 2 - ID: 2729209922
Texto do Anúncio: Luxuoso Apartamento com Elevador Privativo 309,97m privativo 598,93m total 2 totais Descricao: Apres...
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 6.41 segundos
Rolando um pouco a página...
Imóvel já processado: ID 2631303305
Imóvel já processado: ID 2765913054
Imóvel já processado: ID 2765911245
Imóvel já processado: ID 2764996661
Imóvel já processado: ID 2765910908
Imóvel já processado: ID 2765610926
Imóvel já processado: ID 2725011781
Imóvel já processado: ID 2725015669
Imóvel já processado: ID 2763701258
Imóvel já processado: ID 2764996098
Imóvel já processado: ID 2765592237
Imóvel já processado: ID 2765630338
Imóvel já processado: ID 2765915825
Avançando para a próxima página...
Imóvel já processado: ID 2717453185
Imóvel já processado: ID 2729209922
Imóvel já processado: ID 2631303305
Imóvel já processado: ID 2765913054
Imóvel já processad

### Testando ajuste no bloco principal para carregar novos imóveis, pois está sendo fechado o navegador

In [13]:
# URL inicial
start_url = "https://www.zapimoveis.com.br/venda/apartamentos/pr+cascavel/?__ab=exp-aa-test:control,propposv3:control,rentsale:control,pos-zap:new,zapproppos:a&transacao=venda&onde=,Paran%C3%A1,Cascavel,,,,,city,BR%3EParana%3ENULL%3ECascavel,-24.954136,-53.480241,&tipos=apartamento_residencial,studio_residencial,kitnet_residencial,casa_residencial,sobrado_residencial,condominio_residencial,casa-vila_residencial,cobertura_residencial,flat_residencial,loft_residencial&pagina=2"

In [14]:
def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    columns = ["ID", "Códigos", "Área", "Quartos", "Banheiros", "Vagas de Carro", "Preço", "Endereço", "Itens Adicionais", "Texto do Anúncio"]
    df = pd.DataFrame(columns=columns)
    processed_ids = load_progress()  # Carrega IDs processados de arquivo
  
    try:
        driver.get(start_url)
        accept_cookies(driver)

        while True:  # Loop de coleta de páginas
            # Rola até o final da página para carregar todos os imóveis
            print("Rolando gradualmente para carregar todos os anúncios...")
            scroll_to_end(driver)

            # Coleta os imóveis da página atual
            listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
            if not listings:
                print("Nenhum imóvel encontrado nesta página.")
                break

            for i in range(len(listings)):
                try:
                    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
                    link = listings[i].find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    property_id = extract_id_from_url(link)

                    if property_id in processed_ids:
                        print(f"Imóvel já processado: ID {property_id}")
                        continue

                    # Coleta o texto do anúncio
                    try:
                        container = listings[i].find_element(By.XPATH, "./ancestor::div[contains(@class, 'BaseCard_card__content__pL2Vc')]")
                        description_element = container.find_element(By.CSS_SELECTOR, "p.ListingCard_card__description__slBTG[data-cy='rp-cardProperty-description-txt']")
                        raw_text = description_element.text if description_element else "Não encontrado"
                        ad_text = unidecode(raw_text)  # Remove inconsistências de codificação
                    except Exception as e:
                        print(f"Erro ao coletar texto do anúncio no imóvel {i + 1}: {e}")
                        ad_text = "Não encontrado"

                    print(f"\nProcessando imóvel {i + 1} - ID: {property_id}")
                    print(f"Texto do Anúncio: {ad_text[:100]}...")  # Mostra uma parte do texto para depuração

                    processed_ids.add(property_id)  # Adiciona ID ao conjunto de processados
                    save_progress(processed_ids)  # Salva progresso continuamente

                    start_time = time.time()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listings[i])).click()
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])
                    accept_cookies(driver)

                    data = scrape_property_data(driver, property_id, ad_text)
                    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    elapsed_time = time.time() - start_time
                    print(f"Tempo entre clique e retorno: {elapsed_time:.2f} segundos")

                    scroll_a_little(driver)
                except Exception as e:
                    print(f"Erro ao processar imóvel: {e}")

            # Tenta avançar para a próxima página
            if not click_next_page(driver):
                break  # Sai do loop caso não haja mais páginas

            # Após mudar de página, rola novamente para carregar todos os imóveis
            print("Mudou de página, rolando para carregar os anúncios...")
            scroll_to_end(driver)  # Rola na nova página antes de processar

    finally:
        driver.quit()
        df.to_csv("imoveis_zapimoveis.csv", index=False, encoding="utf-8")
        print("Dados salvos no arquivo 'imoveis_zapimoveis.csv'.")

if __name__ == '__main__':
    scrape_zapimoveis()

Cookies aceitos.
Rolando gradualmente para carregar todos os anúncios...
Elemento do rodapé encontrado. Final da página alcançado.
Retornando ao topo da página...
Imóvel já processado: ID 2762614220
Imóvel já processado: ID 2737075447

Processando imóvel 3 - ID: 2733541254
Texto do Anúncio: Se voce esta interessado em uma casa residencial a venda no bairro Esmeralda, com 2 quartos e 1 vaga...
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 6.95 segundos
Rolando um pouco a página...
Erro ao processar imóvel: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./ancestor::a"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0100EC13+23731]
	(No symbol) [0x00F9C394]
	(No symbol) [0x00E7BE63]
	(No symbol) [0x00EBFCE6]
	(No symbol) [0x00EBFF2B]
	(No symbol) [0x00EB

### script em teste 2 

In [35]:
def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    columns = ["ID", "Códigos", "Área", "Quartos", "Banheiros", "Vagas de Carro", "Preço", "Endereço", "Itens Adicionais", "Texto do Anúncio"]
    df = pd.DataFrame(columns=columns)
    processed_ids = load_progress()  # Carrega IDs processados de arquivo

    try:
        driver.get(start_url)
        accept_cookies(driver)

        while True:  # Loop de coleta de páginas
            print("Rolando gradualmente para carregar todos os anúncios...")
            scroll_to_end(driver)  # Rola antes de iniciar a coleta
            
            # Coleta os imóveis da página atual
            listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
            if not listings:
                print("Nenhum imóvel encontrado nesta página.")
                break

            # Verificar se todos os imóveis da página já foram processados
            all_processed = True
            for listing in listings:
                link = listing.find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                property_id = extract_id_from_url(link)
                if property_id not in processed_ids:
                    all_processed = False
                    break

            if all_processed:
                print("Todos os imóveis desta página já foram processados. Avançando para a próxima página...")
                if not click_next_page(driver):
                    break  # Sai do loop se não houver mais páginas
                continue  # Volta ao início do loop para processar a próxima página

            for i in range(len(listings)):
                try:
                    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
                    link = listings[i].find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    property_id = extract_id_from_url(link)

                    if property_id in processed_ids:
                        print(f"Imóvel já processado: ID {property_id}")
                        continue

                    container = listings[i].find_element(By.XPATH, "./ancestor::div[contains(@class, 'BaseCard_card__content__pL2Vc')]")
                    description_element = container.find_element(By.CSS_SELECTOR, "p.ListingCard_card__description__slBTG[data-cy='rp-cardProperty-description-txt']")
                    raw_text = description_element.text if description_element else "Não encontrado"
                    ad_text = unidecode(raw_text)

                    print(f"\nProcessando imóvel {i + 1} - ID: {property_id}")
                    print(f"Texto do Anúncio: {ad_text[:100]}...")

                    processed_ids.add(property_id)
                    save_progress(processed_ids)

                    start_time = time.time()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listings[i])).click()
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])
                    accept_cookies(driver)

                    data = scrape_property_data(driver, property_id, ad_text)
                    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    elapsed_time = time.time() - start_time
                    print(f"Tempo entre clique e retorno: {elapsed_time:.2f} segundos")

                    scroll_a_little(driver)
                except Exception as e:
                    print(f"Erro ao processar imóvel: {e}")

            # Tenta avançar para a próxima página
            print("Preparando para avançar para a próxima página...")
            if not click_next_page(driver):
                break  # Sai do loop se não houver mais páginas

    finally:
        driver.quit()
        df.to_csv("imoveis_zapimoveis.csv", index=False, encoding="utf-8")
        print("Dados salvos no arquivo 'imoveis_zapimoveis.csv'.")
        
if __name__ == '__main__':
    scrape_zapimoveis()

Cookies aceitos.
Rolando gradualmente para carregar todos os anúncios...
Elemento do rodapé encontrado. Final da página alcançado.
Retornando ao topo da página...
Dados salvos no arquivo 'imoveis_zapimoveis.csv'.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./ancestor::a"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x011BEC13+23731]
	(No symbol) [0x0114C394]
	(No symbol) [0x0102BE63]
	(No symbol) [0x0106FCE6]
	(No symbol) [0x0106FF2B]
	(No symbol) [0x01065A81]
	(No symbol) [0x01091EA4]
	(No symbol) [0x010659A4]
	(No symbol) [0x010920F4]
	(No symbol) [0x010AB46E]
	(No symbol) [0x01091BF6]
	(No symbol) [0x01063F35]
	(No symbol) [0x01064EBD]
	GetHandleVerifier [0x0149F0D3+3039603]
	GetHandleVerifier [0x014B2DEA+3120778]
	GetHandleVerifier [0x014AB592+3089970]
	GetHandleVerifier [0x012543B0+635984]
	(No symbol) [0x01154DCD]
	(No symbol) [0x01152068]
	(No symbol) [0x01152205]
	(No symbol) [0x01144FD0]
	BaseThreadInitThunk [0x761E7BA9+25]
	RtlInitializeExceptionChain [

### Nova abordagem

In [25]:
import os
import time
import pandas as pd
import re
from unidecode import unidecode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Pasta para salvar os arquivos CSV
output_dir = r"C:\Users\fluca\Desktop\Data Science\Projetos\projeto_tcc\db_zapimoveis"
os.makedirs(output_dir, exist_ok=True)

# URL base
base_url = "https://www.zapimoveis.com.br/venda/apartamentos/pr+cascavel/?__ab=exp-aa-test:control,propposv3:control,rentsale:control,pos-zap:new,zapproppos:a&transacao=venda&onde=,Paran%C3%A1,Cascavel,,,,,city,BR%3EParana%3ENULL%3ECascavel,-24.954136,-53.480241,&tipos=apartamento_residencial,studio_residencial,kitnet_residencial,casa_residencial,sobrado_residencial,condominio_residencial,casa-vila_residencial,cobertura_residencial,flat_residencial,loft_residencial&pagina={}"

def scroll_to_end(driver):
    """Rola gradualmente até o final da página com base no carregamento dinâmico de anúncios."""
    while True:
        try:
            footer_image = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "img[src='https://cdn-zap-ssr-prod.zapimoveis.com.br/_next/public/assets/images/reclameaqui-stamp.svg']"))
            )
            if footer_image.is_displayed():
                print("Elemento do rodapé encontrado. Final da página alcançado.")
                break
        except Exception:
            print("Elemento do rodapé não encontrado ainda. Continuando a rolagem...")

        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.7)

    print("Retornando ao topo da página...")
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)

def scrape_page(driver):
    """Coleta os dados de todos os anúncios da página atual com delays adequados."""
    properties = []
    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")

    for i, listing in enumerate(listings):
        try:
            print(f"Processando anúncio {i + 1}/{len(listings)}")
            link = listing.find_element(By.XPATH, "./ancestor::a").get_attribute("href")
            property_id = re.search(r"id-(\d+)", link).group(1) if link else f"Unknown_{i}"

            # Coleta o texto do anúncio antes de clicar no link
            try:
                container = listing.find_element(By.XPATH, "./ancestor::div[contains(@class, 'BaseCard_card__content__pL2Vc')]")
                description_element = container.find_element(By.CSS_SELECTOR, "p.ListingCard_card__description__slBTG[data-cy='rp-cardProperty-description-txt']")
                raw_text = description_element.text if description_element else "Não encontrado"
                ad_text = unidecode(raw_text)  # Remove inconsistências de codificação
            except Exception as e:
                print(f"Erro ao coletar texto do anúncio no imóvel {i + 1}: {e}")
                ad_text = "Não encontrado"

            print(f"Texto do Anúncio: {ad_text[:100]}...")  # Mostra uma parte do texto para depuração

            # Clica no anúncio e espera abrir nova aba
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listing)).click()
            WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
            time.sleep(2)  # Aguarda o carregamento inicial da nova aba

            driver.switch_to.window(driver.window_handles[1])  # Troca para a aba do anúncio
            time.sleep(3)  # Tempo adicional para carregamento total

            # Coleta os dados detalhados do imóvel
            data = scrape_property_data(driver, property_id, ad_text)
            properties.append(data)

            driver.close()  # Fecha a aba do anúncio
            driver.switch_to.window(driver.window_handles[0])  # Volta para a aba principal
            time.sleep(1)  # Aguarda estabilização após retornar à aba principal

        except Exception as e:
            print(f"Erro ao processar anúncio {i + 1}: {e}")
            driver.switch_to.window(driver.window_handles[0])  # Garante que estamos na aba principal

    return properties


    return properties

def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)

    for page in range(1, 49):  # De 1 a 48
        print(f"Processando página {page}...")
        url = base_url.format(page)
        driver.get(url)

        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")))
            scroll_to_end(driver)
            data = scrape_page(driver)

            df = pd.DataFrame(data)
            output_file = os.path.join(output_dir, f"imoveis_zapimoveis_{page}.csv")
            df.to_csv(output_file, index=False, encoding="utf-8")
            print(f"Dados da página {page} salvos em: {output_file}")
        except Exception as e:
            print(f"Erro ao processar página {page}: {e}")

    driver.quit()
    print("Coleta concluída!")

if __name__ == "__main__":
    scrape_zapimoveis()


Processando página 1...
Elemento do rodapé encontrado. Final da página alcançado.
Retornando ao topo da página...
Processando anúncio 1/105
Texto do Anúncio: Venha morar com sua familia nesta linda casa no Bairro Parque Verde em Cascavel - Pr. Ambientes acon...
Processando anúncio 2/105
Texto do Anúncio: 1 Suite com closet + 2 quartos, banheiro social; Sala de estar e jantar com cozinha integrada; Edicu...
Processando anúncio 3/105
Texto do Anúncio: Nenhuma palavra define tao bem como: ESPETACULAR! Esse apartamento luxuoso esta localizado em um dos...
Processando anúncio 4/105
Texto do Anúncio: *** RECANTO CANADA RESIDENCIAL *** Com ambientes planejados para todas as idades, a area de lazer co...
Processando anúncio 5/105
Texto do Anúncio: SOBRADOS GEMINADOS COM 64M2 DE AREA PRIVATIVA, EM CONDOMINIO FECHADO, COM AREA DE LAZER E SEGURANCA;...
Processando anúncio 6/105
Texto do Anúncio: RESIDENCIAL COQUEIRAL - RUA DAS PALMEIRAS - CASCAVEL-PR. CONDOMINIO COM 07 BLOCOS DE APARTAMENTOS. A..

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x004DEC13+23731]
	(No symbol) [0x0046C394]
	(No symbol) [0x0034BE63]
	(No symbol) [0x0032D92B]
	(No symbol) [0x003B7F7F]
	(No symbol) [0x003CAD99]
	(No symbol) [0x003B1BF6]
	(No symbol) [0x00383F35]
	(No symbol) [0x00384EBD]
	GetHandleVerifier [0x007BF0D3+3039603]
	GetHandleVerifier [0x007D2DEA+3120778]
	GetHandleVerifier [0x007CB592+3089970]
	GetHandleVerifier [0x005743B0+635984]
	(No symbol) [0x00474DCD]
	(No symbol) [0x00472068]
	(No symbol) [0x00472205]
	(No symbol) [0x00464FD0]
	BaseThreadInitThunk [0x761E7BA9+25]
	RtlInitializeExceptionChain [0x77B4C0CB+107]
	RtlClearBits [0x77B4C04F+191]


### Imóveis com anúncios duplicados

In [ ]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
def scroll_until_element(driver, element_selector):
    """Rola lentamente a página até que o elemento especificado apareça e clique nele."""
    print("Rolando a página para encontrar o botão de anúncios duplicados...")
    while True:
        try:
            # Verificar se o botão está visível na página
            button = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, element_selector))
            )
            print("Botão de anúncios duplicados encontrado. Clicando...")
            button.click()
            break
        except TimeoutException:
            # Continuar rolando caso o botão não seja encontrado
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(1)

In [ ]:
def interact_with_sidebar(driver):
    """Interage com os elementos dentro da aba lateral e coleta informações."""
    print("Interagindo com a aba lateral...")
    collected_data = []
    try:
        # Aguardar até que os elementos de preço estejam disponíveis
        price_elements = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h2.MainValue_advertiser__total__1ornY"))
        )
        for idx, element in enumerate(price_elements):
            print(f"\tClicando no elemento {idx + 1} com preço: {element.text}")
            ActionChains(driver).move_to_element(element).click().perform()
            time.sleep(2)  # Simula interação humana

            # Alterna para a nova guia
            driver.switch_to.window(driver.window_handles[-1])
            print(f"\tNova guia aberta: {driver.current_url}")

            # Extrair ID do imóvel e dados da propriedade
            property_id = extract_id_from_url(driver.current_url)
            property_data = scrape_property_data(driver, property_id)
            collected_data.append(property_data)

            # Fecha a guia atual
            driver.close()
            print("\tGuia fechada.")

            # Retorna para a guia original
            driver.switch_to.window(driver.window_handles[0])
            print("\tRetornando para a aba principal.")
    except TimeoutException:
        print("Elementos de preço não encontrados na aba lateral.")
    finally:
        print("Interação com a aba lateral concluída.")
    return collected_data

In [ ]:
def main():
    # Configurar o driver com undetected_chromedriver
    options = uc.ChromeOptions()
    options.headless = False  # Não ocultar o navegador para debug, pode ser alterado para True
    driver = uc.Chrome(options=options)

    driver.get("https://www.zapimoveis.com.br/venda/apartamentos/pr+cascavel/?pagina=1")
    accept_cookies(driver)
    driver.maximize_window()

    try:
        # Rola a página até encontrar e clicar no botão de anúncios duplicados
        scroll_until_element(driver, "button[data-cy='listing-card-deduplicated-button']")

        # Interage com a aba lateral e coleta os dados
        property_data_list = interact_with_sidebar(driver)

        # Itera e imprime os dados coletados
        print("\nDados coletados:")
        for property_data in property_data_list:
            print(property_data)
    except Exception as e:
        print(f"Erro durante a execução: {e}")
    finally:
        driver.quit()
        print("Script finalizado.")

if __name__ == "__main__":
    main()